In [1]:
import json
import requests
import pandas as pd
import os
from bs4 import BeautifulSoup

In [2]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [3]:
from langchain_community.utilities import GoogleSerperAPIWrapper
search = GoogleSerperAPIWrapper()

In [4]:
r = requests.get("https://pan-unit42.github.io/playbook_viewer/playbooks.json")

index = []
if r.status_code == 200:
    print("Loading playbooks")
    
    pb_list = r.json()
    
    for report in pb_list:
        #print(f"Title: {report['title']} File Name: {report['pb_file']}")
        stix_name = report['pb_file']
        
        #search pan unit 42 block
        r_search = requests.get(f"https://www.paloaltonetworks.com/search/unit42search#q={report['title']}")
        
        if r_search.status_code == 200:
            print(r_search.url)
            soup = BeautifulSoup(r_search.content, "html.parser")
            titles = soup.find_all("a")

        results = search.results(f"{report['title']} report from Unit42")

        blog_found = False
        if 'organic' in results:
            for result in results['organic']:
                if 'unit42' in result['link']:
                    index.append({"Title":report['title'],"File Name":stix_name,"Blog Title":result['title'],"Blog URL":result['link']})  
                    blog_found = True
                    
        if blog_found == False:
            # could not find a blog
            index.append({"Title":report['title'],"File Name":stix_name,"Blog Title":None,"Blog URL":None})
        
        if os.path.isfile(stix_name): continue
        
        with open(report['pb_file'],'w') as file:
            headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
            rf = requests.get(f"https://pan-unit42.github.io/playbook_viewer/playbook_json/{stix_name}",headers=headers)
             
            if r.status_code == 200:
                json.dump(rf.json(),file)
            else:
                print(r.status_code)
                
    index_df = pd.DataFrame(index)
    
    print("Done!")

Loading playbooks
https://www.paloaltonetworks.com/search/unit42search#q=EVASIVE%20SERPENS
https://www.paloaltonetworks.com/search/unit42search#q=FIGHTING%20URSA
https://www.paloaltonetworks.com/search/unit42search#q=PRYING%20LIBRA
https://www.paloaltonetworks.com/search/unit42search#q=OBSCURE%20SERPENS
https://www.paloaltonetworks.com/search/unit42search#q=MOLDY%20PISCES
https://www.paloaltonetworks.com/search/unit42search#q=RANCOR%20TAURUS
https://www.paloaltonetworks.com/search/unit42search#q=STALKER%20TAURUS
https://www.paloaltonetworks.com/search/unit42search#q=SHALLOW%20TAURUS
https://www.paloaltonetworks.com/search/unit42search#q=GRANITE%20TAURUS
https://www.paloaltonetworks.com/search/unit42search#q=IRON%20TAURUS
https://www.paloaltonetworks.com/search/unit42search#q=BOGGY%20SERPENS
https://www.paloaltonetworks.com/search/unit42search#q=RADIO%20SERPENS
https://www.paloaltonetworks.com/search/unit42search#q=AGED%20LIBRA
https://www.paloaltonetworks.com/search/unit42search#q=MULE

In [5]:
index_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       299 non-null    object
 1   File Name   299 non-null    object
 2   Blog Title  280 non-null    object
 3   Blog URL    280 non-null    object
dtypes: object(4)
memory usage: 9.5+ KB


In [6]:
index_df.head(4)

,Title,File Name,Blog Title,Blog URL
0,EVASIVE SERPENS,evasive-serpens.json,Agonizing Serpens (Aka Agrius) Targeting the I...,https://unit42.paloaltonetworks.com/agonizing-...
1,EVASIVE SERPENS,evasive-serpens.json,pan-unit42/playbook_viewer - GitHub,https://github.com/pan-unit42/playbook_viewer
2,EVASIVE SERPENS,evasive-serpens.json,Than Meets the Eye: Exposing a Polyglot File T...,https://unit42.paloaltonetworks.com/polyglot-f...
3,FIGHTING URSA,fighting-ursa.json,Fighting Ursa Aka APT28: Illuminating a Covert...,https://unit42.paloaltonetworks.com/russian-ap...


In [7]:
index_df.tail(4)

,Title,File Name,Blog Title,Blog URL
295,MUDDLED LIBRA,muddled-libra.json,Phishing Archives - Unit 42,https://unit42.paloaltonetworks.com/tag/phishing/
296,MUDDLED LIBRA,muddled-libra.json,Resilient Cybersecurity Tactics for CISOs - Pa...,https://www.paloaltonetworks.com.au/resources/...
297,MUDDLED LIBRA,muddled-libra.json,Scattered Spider Archives - Unit 42 - Palo Alt...,https://unit42.paloaltonetworks.com/tag/scatte...
298,MUDDLED LIBRA,muddled-libra.json,Unit 42 Threat Intel Bulletin - August - LinkedIn,https://www.linkedin.com/pulse/unit-42-threat-...


In [8]:
index_df.to_csv("pan_unit42.csv",index=False)